In [1]:
!pip install lightning
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.5 MB/s eta 0:00:00


In [2]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import numpy as np
import pandas as pd
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import os
import random
from glob import glob
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
from torchvision import disable_beta_transforms_warning
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader

from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights

import lightning as L
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.loggers import CSVLogger, WandbLogger

RANDOM_SEED = 42
seed_everything(seed=RANDOM_SEED,workers=True)

# Transforms ===============================================

def get_valid_transform(image_size, precision='32-true'):
    if '16' in precision:
        dtype = torch.float16
    elif '32' in precision:
        dtype = torch.float32
    elif '64' in precision:
        dtype = torch.float64
    else:
        raise NotImplementedError(f'{precision} not implemented')
    
    valid_transform = v2.Compose([
        v2.PILToTensor(),
        v2.Resize((image_size, image_size),antialias=True),
        v2.ConvertImageDtype(dtype=dtype),
        v2.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])
    ])
    return valid_transform

# Datasets ===========================================================
class MyDataset(Dataset):
    def __init__(self, root, df, transform):
        super(MyDataset).__init__()
        self.root = root
        self.paths = list(df['path'])
        self.transform = transform
        
    def __repr__(self):
        out = super().__repr__()
        return f"{out} of size {self.__len__()}\n{self.transform.__repr__()}"
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        # get dtype conversion from the transform
        dtype = torch.float32
        for t in self.transform.transforms:
            if isinstance(t,v2.ConvertImageDtype):
                dtype = t.dtype
        img = Image.open(path)
            
        if self.transform:
            img = self.transform(img)
        
        return img, path

#===============================================
# helper function    
# model loader
def get_model_from_wandb(logger, artifact_name, device, save_dir_root='/kaggle/working'):
    """
    model artifact will be downloaded as /{save_dir_root}/{run_id}/model.ckpt
    """
    # download artifact to /kaggle/working
    run_id = artifact_name.split('/')[2]
    logger.download_artifact(artifact_name, save_dir=f"{save_dir_root}/{run_id}", artifact_type='model')
    ckpt_path = sorted(glob(f"{save_dir_root}/{run_id}/*.ckpt"))[-1]
    return LitEfficientNet.load_from_checkpoint(ckpt_path, map_location=device)
    
#===============================================
# Lightning Module

class LitEfficientNet(L.LightningModule):
    def __init__(self, model_config):
        '''
        parameters:
        model_config: model configurations required to initialize lightning model
        '''
        super().__init__()
        self.save_hyperparameters()
        self.model = efficientnet_v2_s(weights = EfficientNet_V2_S_Weights)
        self.model.classifier[1] = \
            nn.Linear(in_features=1280, out_features=1, bias=True)
        # self.model.half
        self.sigmoid = nn.Sigmoid()
    
    def predict_step(self,batch,batch_idx):
        x, path = batch
        pred = self.model(x)
        pred = self.sigmoid(pred)
        return path, pred

INFO: Seed set to 42


In [4]:
# create metadata.csv
ROOT = '/kaggle/input/famous-fake-faces'
path = glob(f'{ROOT}/*/*.??g')
split = [p.split('/')[4] for p in path]
df_meta = pd.DataFrame.from_dict({
    'path' : path,
    'split' : split
})
df_meta.to_csv('output.csv',index=False)

# configs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_config = {
    'root' : '/kaggle/input/using-yunet',
    'image_size' : 224,
    'batch_size' : 16,
    'num_workers' : 3
}

artifacts = [
    # TODO copy paste artifact api ids
    # a. gb0.1 
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2404021253_fold_0:v0',
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2404021319_fold_1:v0',
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2404021345_fold_2:v0',
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2404021412_fold_3:v0',
    # b. gb0.1-re0.2:1
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2403240728_fold_0:v0',
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2403240757_fold_1:v0',
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2403240826_fold_2:v0',
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2403240854_fold_3:v0',
    # c. gb0.1-re0.2:2
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2403241057_fold_0:v0',
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2403241124_fold_1:v0',
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2403241150_fold_2:v0',
    'luanakwon/Deepfake_Detection-lightning-4cv/model-2403241217_fold_3:v0'
]
trained_group = [
    'gb0.1', 'gb0.1-re0.2:1', 'gb0.1-re0.2:2'
]

In [5]:
# prep dataset and dataloader
dset = MyDataset(data_config['root'], 
                 df_meta,get_valid_transform(
                     image_size=data_config['image_size']))
loader = DataLoader(dataset=dset, 
                    batch_size=data_config['batch_size'],
                    num_workers=data_config['num_workers'],
                    shuffle=False)
print(dset)

<__main__.MyDataset object at 0x7db52f01df00> of size 345
Compose(
      PILToTensor()
      Resize(size=[224, 224], interpolation=InterpolationMode.BILINEAR, antialias=True)
      ConvertImageDtype()
      Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=False)
)


In [6]:
# iterate through given artifacts
for a_idx, artifact_id in enumerate(artifacts):
    group_name=trained_group[a_idx//4]
    # download artifact if not exist at /kaggle/working & init model
    run_id = artifact_id.split('/')[-1]
    fold_id = int(run_id.split(':')[0].split('_fold_')[-1])
    if not os.path.exists(f'/kaggle/working/artifacts/{run_id}'):
        model = get_model_from_wandb(
            WandbLogger(), artifact_id, device, save_dir_root = '/kaggle/working/artifacts')
    else:
        print(f"Artifact '{artifact_id}' already exists. Using existing file...")
        ckpt_path = sorted(glob(f"/kaggle/working/artifacts/{run_id}/*.ckpt"))[-1]
        model = LitEfficientNet.load_from_checkpoint(ckpt_path, map_location=device)
        
#     # init logger
#     logger = CSVLogger(save_dir='/kaggle/working/results',
#                        name=f'{group_name}_f{fold_id}')

    # model predict
    trainer = Trainer(accelerator=device.type)
    predictions = trainer.predict(model=model, dataloaders=loader)

    df_tmp = pd.DataFrame.from_dict(
        {
            'path': [pth for pred in predictions for pth in pred[0]],
             f'pred_{group_name}_f{fold_id}': 
                [p.item() for pred in predictions for p in pred[1]]
        }
    )
    print(df_tmp)
    df_left = pd.read_csv('/kaggle/working/output.csv')
    df_left = pd.merge(df_left,df_tmp,left_on='path',right_on='path')
    df_left.to_csv('output.csv',index=False)
    print(df_left)

wandb: Downloading large artifact model-2404021253_fold_0:v0, 77.68MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.1
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 133MB/s]
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
2024-05-06 09:56:37.189106: E external/local_xla/xla/stream_executo

Predicting: |          | 0/? [00:00<?, ?it/s]

                                                  path  pred_gb0.1_f0
0    /kaggle/input/famous-fake-faces/etc/TomHolland...       0.055642
1    /kaggle/input/famous-fake-faces/etc/BarackObam...       0.000104
2    /kaggle/input/famous-fake-faces/etc/TomCruise_...       0.716359
3    /kaggle/input/famous-fake-faces/etc/yilongma_0...       0.002342
4    /kaggle/input/famous-fake-faces/etc/RobertDown...       0.134128
..                                                 ...            ...
340  /kaggle/input/famous-fake-faces/yilongma_short...       0.028924
341  /kaggle/input/famous-fake-faces/yilongma_short...       0.041639
342  /kaggle/input/famous-fake-faces/yilongma_short...       0.029501
343  /kaggle/input/famous-fake-faces/yilongma_short...       0.006423
344  /kaggle/input/famous-fake-faces/yilongma_short...       0.101164

[345 rows x 2 columns]
                                                  path              split  \
0    /kaggle/input/famous-fake-faces/etc/TomHolland...     

wandb: Downloading large artifact model-2404021319_fold_1:v0, 77.68MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.3
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

                                                  path  pred_gb0.1-re0.2:2_f2
0    /kaggle/input/famous-fake-faces/etc/TomHolland...               0.008969
1    /kaggle/input/famous-fake-faces/etc/BarackObam...               0.019113
2    /kaggle/input/famous-fake-faces/etc/TomCruise_...               0.325775
3    /kaggle/input/famous-fake-faces/etc/yilongma_0...               0.006485
4    /kaggle/input/famous-fake-faces/etc/RobertDown...               0.109056
..                                                 ...                    ...
340  /kaggle/input/famous-fake-faces/yilongma_short...               0.238937
341  /kaggle/input/famous-fake-faces/yilongma_short...               0.087868
342  /kaggle/input/famous-fake-faces/yilongma_short...               0.012005
343  /kaggle/input/famous-fake-faces/yilongma_short...               0.005471
344  /kaggle/input/famous-fake-faces/yilongma_short...               0.028228

[345 rows x 2 columns]
                                        

wandb: Downloading large artifact model-2403241217_fold_3:v0, 77.68MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.6
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

                                                  path  pred_gb0.1-re0.2:2_f3
0    /kaggle/input/famous-fake-faces/etc/TomHolland...               0.003730
1    /kaggle/input/famous-fake-faces/etc/BarackObam...               0.016752
2    /kaggle/input/famous-fake-faces/etc/TomCruise_...               0.024053
3    /kaggle/input/famous-fake-faces/etc/yilongma_0...               0.000714
4    /kaggle/input/famous-fake-faces/etc/RobertDown...               0.015372
..                                                 ...                    ...
340  /kaggle/input/famous-fake-faces/yilongma_short...               0.005853
341  /kaggle/input/famous-fake-faces/yilongma_short...               0.003949
342  /kaggle/input/famous-fake-faces/yilongma_short...               0.002319
343  /kaggle/input/famous-fake-faces/yilongma_short...               0.002391
344  /kaggle/input/famous-fake-faces/yilongma_short...               0.002212

[345 rows x 2 columns]
                                        

In [7]:
df_out = pd.read_csv('/kaggle/working/output.csv')
df_out

,path,split,pred_gb0.1_f0,pred_gb0.1_f1,pred_gb0.1_f2,pred_gb0.1_f3,pred_gb0.1-re0.2:1_f0,pred_gb0.1-re0.2:1_f1,pred_gb0.1-re0.2:1_f2,pred_gb0.1-re0.2:1_f3,pred_gb0.1-re0.2:2_f0,pred_gb0.1-re0.2:2_f1,pred_gb0.1-re0.2:2_f2,pred_gb0.1-re0.2:2_f3
0,/kaggle/input/famous-fake-faces/etc/TomHolland...,etc,0.055642,0.007047,0.008439,0.000733,0.002975,0.001405,0.012260,0.001224,0.002373,0.000977,0.008969,0.003730
1,/kaggle/input/famous-fake-faces/etc/BarackObam...,etc,0.000104,0.000177,0.001046,0.002905,0.000215,0.004184,0.003430,0.002772,0.006017,0.024789,0.019113,0.016752
2,/kaggle/input/famous-fake-faces/etc/TomCruise_...,etc,0.716359,0.750805,0.735833,0.051428,0.018028,0.186970,0.101960,0.031023,0.087207,0.079809,0.325775,0.024053
3,/kaggle/input/famous-fake-faces/etc/yilongma_0...,etc,0.002342,0.000962,0.003152,0.000095,0.000207,0.005981,0.002074,0.000064,0.000968,0.001658,0.006485,0.000714
4,/kaggle/input/famous-fake-faces/etc/RobertDown...,etc,0.134128,0.039044,0.007441,0.048399,0.069316,0.035239,0.015940,0.021074,0.171696,0.022874,0.109056,0.015372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,/kaggle/input/famous-fake-faces/yilongma_short...,yilongma_shorts_1,0.028924,0.026598,0.081606,0.000396,0.052774,0.022596,0.287521,0.366744,0.018099,0.009576,0.238937,0.005853
341,/kaggle/input/famous-fake-faces/yilongma_short...,yilongma_shorts_1,0.041639,0.005180,0.153586,0.005674,0.043880,0.033399,0.152263,0.050526,0.011456,0.000941,0.087868,0.003949
342,/kaggle/input/famous-fake-faces/yilongma_short...,yilongma_shorts_1,0.029501,0.002340,0.050821,0.006441,0.000533,0.001168,0.024154,0.001156,0.001902,0.001703,0.012005,0.002319
343,/kaggle/input/famous-fake-faces/yilongma_short...,yilongma_shorts_1,0.006423,0.002588,0.010351,0.000729,0.003398,0.004499,0.018400,0.001902,0.001706,0.001265,0.005471,0.002391


In [8]:
for col_name in df_out.columns[2:]:
    print(col_name)
    print(f"{np.mean(df_out[col_name])}")

pred_gb0.1_f0
0.18519568028895647
pred_gb0.1_f1
0.025314887215569317
pred_gb0.1_f2
0.08696622332267837
pred_gb0.1_f3
0.025781394991904977
pred_gb0.1-re0.2:1_f0
0.022489165421406544
pred_gb0.1-re0.2:1_f1
0.026860695390833786
pred_gb0.1-re0.2:1_f2
0.05369977547949095
pred_gb0.1-re0.2:1_f3
0.020731130851630034
pred_gb0.1-re0.2:2_f0
0.021433511165525727
pred_gb0.1-re0.2:2_f1
0.02395295959864013
pred_gb0.1-re0.2:2_f2
0.05355322645887293
pred_gb0.1-re0.2:2_f3
0.011093613018021883


In [9]:
!rm -r /kaggle/working/lightning_logs
!rm -r /kaggle/working/artifacts
# TODO
# plot pred * group
# plot pred*group per category(split)
# plot image, pred*group of 5 worst